# Model Isinga 2D
### Dominik Stańczak
Symulacja dwuwymiarowego modelu Isinga - modelu ferromagnetyka na prostokątnej siatce spinów. Każdy spin ulega losowemu odwracaniu między górą a dołem z prawdopodobieństwem z rozkładu Boltzmanna, za energie przyjmując iloczyn własnego spinu i spinów czterech najbliższych sąsiadów, oraz wkład od zewnętrznego pola (dalej zwanego magnetyzacją).

Kod źródłowy symulacji można znaleźć [na www.github.com/StanczakDominik/ising](https://github.com/StanczakDominik/ising/blob/master/Ising-SSN.ipynb), zaś do uruchomienia jej polecam [dystrybucję Anaconda Pythona 3.5 dostępną na www.continuum.io/downloads](https://www.continuum.io/downloads).

## Od strony teoretycznej:

Energia pojedynczego spinu:
$$E_i = -S_i \Big(\sum_{\text{sąsiedzi }j} J_{ij} S_j + \mu H_i\Big) $$

Przyjmujemy dla uproszczenia, że wszystkie oddziaływania są takie same:
$$ J_{ij} = J $$
Prawdopodobieństwo stanu z daną energią (rozkład kanoniczny):
$$P(E) = Z \exp(-\beta E) = Z \exp(-E / k_BT)$$
Prawdopodobieństwo przejścia ze stanu 1 do stanu 2:
$$P(E_1 \to E_2) = \max\Big(\frac{P(E_2)}{P(E_1)}, 1\Big) = \max\Big(\exp(-\beta(E_2-E_1)), 1\Big)$$
Jeśli za stany przyjmiemy stany pojedynczych spinów, z energiami określonymi przez energię na siatce, zaś zmiana stanu to obrócenie jednego spinu w przeciwną stronę, czyli $S_i^2 = -S_i^1$:
$$ E_2 - E_1 \sim -S_i^2 + S_i^1 = 2 S_i ^1$$
$$ \Delta E = 2 S_i \Big(\sum_{\text{sąsiedzi }j} J_{ij} S_j + \mu H_i\Big) $$

Na tej podstawie możemy policzyć prawdopodobieństwo przejścia dowolnie wybranego spinu.

### Parametry warte prześledzenia:
* $T = 1, H = 0$ - ferromagnetyk z magnetyzacją spontaniczną
* $T = 1, H \neq 0$ - ferromagnetyk w zewnętrznym polu
* $T = 0.1$ - ferromagnetyk z niskim szumem termicznym
* $T = 2$ - ferromagnetyk z wysokim szumem termicznym
* $T > 2.26$ ($T \sim 10$) - paramagnetyk
* $T \sim 2.26$ - przejście fazowe (układ zwalnia i warto zwiększyć $NT = 10^6$)
* $T = 3, H = 1$ - paramagnetyk w stosunkowo silnym polu
* $T = 3, H = 500$ - paramagnetyk w wyjątkowo silnym polu

### Uruchamianie symulacji
Należy kliknąć komórkę z kodem poniżej oraz wcisnąć `Ctrl-Enter` (`Shift-Enter`), by od razu przejść do wyników (obliczanie może chwilę trwać).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, animation

T = 1     #temperatura. Temperatura krytyczna w bezwymiarowych jednostkach (k, J = 1): ~2.26
H = 0.0   #wartość magnetyzacji 
N = 32    #szerokość siatki (rozmiar siatki N*N)

k = 1     #stała Boltzmanna
J = 1     #całka wymiany - sprzężenie spinu ze spinem
mu = 1    #sprzężenie spinu i magnetyzacji

NT = 10000                        #liczba iteracji czasowych
N_snapshots = 100                 #liczba zdjęć
snap_every = NT // N_snapshots    #co ile iteracji robić zdjęcie

H_array = np.ones((N, N)) * H                 #magnetyzacja niezależna od położenia, aby ustawić zależną...
# X, Y = np.meshgrid(np.arange(N) - N/2, np.arange(N) - N/2) #pomocnicze macierze
# H_array = -H*np.exp((-X*X-Y-Y)/100)         #przykład magnetyzacji zależnej od położenia

 
# symulacja jest losowa, jeśli chcemy ustalić warunki początkowe należy zaseedować generator liczb losowych
# np.random.seed(1)

############## "wnętrze" symulacji
rc('animation', html='html5')
rc('figure', max_open_warning=500)

spins = np.random.randint(0, 2, size=(N, N)) * 2 - 1
def flip_spin(spins):
    x, y = np.random.randint(0, N, size=2)
    beta = 1/k/T
    energia = 2 * spins[x,y] *\
        (J* (spins[(x+1)%N, y] + spins[(x-1)%N, y] + spins[x, (y+1)%N] + spins[x, (y-1)%N]) +\
         mu * H_array[x,y]) 
    boltzmann = np.exp(-energia*beta)
    acceptance = np.random.random()
    if boltzmann > acceptance:
        spins[x, y] *= -1

spins_history = np.empty(shape=(N_snapshots, N, N))
def petla_czasowa():
    for i in range(NT):
        if i % snap_every == 0:
            spins_history[i // snap_every] = spins
        flip_spin(spins)

petla_czasowa()

def animacja(spins):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    IM = ax.imshow(spins_history[0], origin='lower', cmap='Greys',
                  interpolation='none', extent=(-N/2, N/2, -N/2, N/2))
    title = ax.set_title("T= {}".format(T) + " i = 0 ")
    def animate(i):
        IM.set_array(spins_history[i])
        title.set_text("T= {}".format(T) + " i = {} ".format(i*snap_every))
        return [IM, title]

    anim = animation.FuncAnimation(fig, animate, frames = N_snapshots)
    return anim
animacja(spins)